[Reference](https://medium.com/@adalseno/python-at-the-speed-of-light-216700cf8469)

In [1]:
def populate_python(size:int)->list:
    b = []
    for i in range(size):
        b.append(i)
    return b

In [2]:
import numpy as np
def populate_numpy(size:int)->np.ndarray:
    b = np.empty((size),dtype=np.int64)
    for i in range(size):
        b[i] = i
    return b

In [3]:
from numba import njit, prange
@njit
def populate_numba(size:int)->np.ndarray:
    b = np.empty((size),dtype=np.int64)
    for i in prange(size):
        b[i] = i
    return b

In [5]:
# function populate_array(size::Int)::AbstractVector{Int64}
#     b = Vector{Int64}(undef,size)
#     Threads.@threads for i=1:size
#         b[i] = i
#     end
#     return b
# end

In [6]:
from Pointer import DTypePointer
from Random import rand, random_ui64
from DType import DType
from Range import range
from Functional import parallelize
import SIMD

struct Vect:
    var data: DTypePointer[DType.uint64]
    var rows: Int

    fn __init__(inout self, rows: Int):
        self.data = DTypePointer[DType.uint64].alloc(rows)
        self.rows = rows

    fn __del__(owned self):
        self.data.free()

    @always_inline
    fn len(self)->UInt64:
        return self.rows

    fn zero(inout self):
        memset_zero(self.data, self.rows)

    @always_inline
    fn __getitem__(self, x: Int) -> UInt64:
        return self.data.load(x)



    @always_inline
    fn __setitem__(self,  x: Int, val: UInt64):
        return self.data.store( x, val)

fn populate_mojo(b:Vect):
    @parameter
    fn process_row(i:Int):
        b[i] = i
    parallelize[process_row](b.rows)